# LatNetBuilder

This demo shows how to use generating vectors and matricies from [latnetbuilder](https://github.com/umontreal-simul/latnetbuilder) in QMCPy. 

In [1]:
import latnetbuilder
from qmcpy import latnetbuilder_linker, Lattice, Sobol
import numpy
numpy.set_printoptions(threshold=2**10)
import os

In [2]:
# make directory to store generating vectors + matricies from latnetbuilder
try:
    os.mkdir('./lnb')
    os.mkdir('./lnb/lnb_qmcpy_linked')
except: pass

## Ordinary Lattice

In [3]:
# use latnetbuilder to find an ordinary lattice generating vector 
search = latnetbuilder.SearchLattice()
search.modulus = '2^3'
search.construction = 'ordinary'
search.dimension = 5
search.exploration_method = 'fast-CBC'
search.figure_of_merit = 'CU:P2'
search.weights = ['product:1']
search.execute(output_folder='./lnb/lnb_ordinary_lattice')

Result:
Number of points: 8
Generating Vector: [1, 3, 1, 1, 1]
Merit value: 178.882 
CPU Time: 0.000621109 seconds


In [4]:
# get points using latnetbuilder
x_lnb = search.points()
x_lnb

array([[0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.125, 0.375, 0.125, 0.125, 0.125],
       [0.25 , 0.75 , 0.25 , 0.25 , 0.25 ],
       [0.375, 0.125, 0.375, 0.375, 0.375],
       [0.5  , 0.5  , 0.5  , 0.5  , 0.5  ],
       [0.625, 0.875, 0.625, 0.625, 0.625],
       [0.75 , 0.25 , 0.75 , 0.75 , 0.75 ],
       [0.875, 0.625, 0.875, 0.875, 0.875]])

In [5]:
# store generating vector in QMCPy format
lnb_ol_p = latnetbuilder_linker(
    lnb_dir = './lnb/lnb_ordinary_lattice',
    out_dir = './lnb/lnb_qmcpy_linked',
    fout_prefix = 'lnb_ordinary_lattice')
lnb_ol_p # path to generating vector compatible with QMCPy

'./lnb/lnb_qmcpy_linked/lnb_ordinary_lattice.5.3.npy'

In [6]:
# use the custom generating vector in QMCPy
lnb_lattice = Lattice(
    dimension = 5,
    randomize = False,
    order = 'linear',
    z_path = lnb_ol_p) # plug in the path
x_qmcpy = lnb_lattice.gen_samples(
    n_min = 0,
    n_max = 8, 
    warn = False)
x_qmcpy

array([[0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.125, 0.375, 0.125, 0.125, 0.125],
       [0.25 , 0.75 , 0.25 , 0.25 , 0.25 ],
       [0.375, 0.125, 0.375, 0.375, 0.375],
       [0.5  , 0.5  , 0.5  , 0.5  , 0.5  ],
       [0.625, 0.875, 0.625, 0.625, 0.625],
       [0.75 , 0.25 , 0.75 , 0.75 , 0.75 ],
       [0.875, 0.625, 0.875, 0.875, 0.875]])

In [7]:
# make sure the two matricies match
if not (x_lnb==x_qmcpy).all():
    raise Exception("Ordinary Lattice from latnetbuilder does not match QMCPy")

## Polynomial Lattice

In [8]:
search = latnetbuilder.SearchLattice()
search.modulus = '2^3'
search.construction = 'polynomial'
search.dimension = 5
search.exploration_method = 'fast-CBC'
search.figure_of_merit = 'CU:P2'
search.weights = ['product:1']
search.execute(output_folder='./lnb/lnb_polynomial_lattice')

Result:
Number of points: 8
Modulus: [1, 1, 0, 1]
Generating Vector: [[1], [0, 0, 1], [0, 1], [1], [1]]
Merit value: 29.375 
CPU Time: 0.000928431 seconds


In [9]:
x_lnb = search.points()
x_lnb

array([[0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.125, 0.625, 0.25 , 0.125, 0.125],
       [0.25 , 0.375, 0.625, 0.25 , 0.25 ],
       [0.375, 0.75 , 0.875, 0.375, 0.375],
       [0.625, 0.875, 0.375, 0.625, 0.625],
       [0.5  , 0.25 , 0.125, 0.5  , 0.5  ],
       [0.875, 0.5  , 0.75 , 0.875, 0.875],
       [0.75 , 0.125, 0.5  , 0.75 , 0.75 ]])

In [10]:
lnb_pl_p = latnetbuilder_linker(
    lnb_dir = './lnb/lnb_polynomial_lattice',
    out_dir = './lnb/lnb_qmcpy_linked',
    fout_prefix = 'lnb_polynomial_lattice')
lnb_pl_p

'./lnb/lnb_qmcpy_linked/lnb_polynomial_lattice.5.3.msb.npy'

In [11]:
# note that we use QMCPy's Sobol' object rather than Lattice 
# this is because polynomial lattices are constructed like digital nets
lnb_pl = Sobol(
    dimension = 5,
    randomize = False,
    graycode = False,
    z_path = lnb_pl_p)
x_qmcpy = lnb_pl.gen_samples(
    n_min = 0,
    n_max = 8,
    warn = False)

In [12]:
if not (x_lnb==x_qmcpy).all():
    raise Exception("Ordinary Lattice from latnetbuilder does not match QMCPy")

## Sobol'

In [13]:
search = latnetbuilder.SearchNet()
search.modulus = '2^3'
search.construction = 'sobol'
search.dimension = 5
search.exploration_method = 'full-CBC'
search.figure_of_merit = 'CU:P2'
search.weights = ['product:1']
search.execute(output_folder='./lnb/lnb_sobol')

Result:
Number of points: 8
Direction numbers: [[0], [1], [1, 1], [1, 1, 1], [1, 1, 1]]
Merit value: 29.375 
CPU Time: 0.00064445 seconds


In [14]:
x_lnb = search.points()
x_lnb

array([[0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.5  , 0.5  , 0.5  , 0.5  , 0.5  ],
       [0.25 , 0.75 , 0.25 , 0.25 , 0.25 ],
       [0.75 , 0.25 , 0.75 , 0.75 , 0.75 ],
       [0.125, 0.625, 0.875, 0.125, 0.125],
       [0.625, 0.125, 0.375, 0.625, 0.625],
       [0.375, 0.375, 0.625, 0.375, 0.375],
       [0.875, 0.875, 0.125, 0.875, 0.875]])

In [15]:
lnb_s_p = latnetbuilder_linker(
    lnb_dir = './lnb/lnb_sobol',
    out_dir = './lnb/lnb_qmcpy_linked',
    fout_prefix = 'lnb_sobol')
lnb_s_p

'./lnb/lnb_qmcpy_linked/lnb_sobol.5.3.msb.npy'

In [16]:
lnb_sobol = Sobol(
    dimension = 5,
    randomize = False,
    graycode = False,
    z_path = lnb_s_p)
x_qmcpy = lnb_sobol.gen_samples(
    n_min = 0,
    n_max = 8,
    warn = False)
x_qmcpy

array([[0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.5  , 0.5  , 0.5  , 0.5  , 0.5  ],
       [0.25 , 0.75 , 0.25 , 0.25 , 0.25 ],
       [0.75 , 0.25 , 0.75 , 0.75 , 0.75 ],
       [0.125, 0.625, 0.875, 0.125, 0.125],
       [0.625, 0.125, 0.375, 0.625, 0.625],
       [0.375, 0.375, 0.625, 0.375, 0.375],
       [0.875, 0.875, 0.125, 0.875, 0.875]])

In [17]:
if not (x_lnb==x_qmcpy).all():
    raise Exception("Ordinary Lattice from latnetbuilder does not match QMCPy")

## Output Directories

In [18]:
# list output vectors / matricies
os.listdir('./lnb/lnb_qmcpy_linked')

['lnb_sobol.5.3.msb.npy',
 'lnb_ordinary_lattice.5.3.npy',
 'lnb_polynomial_lattice.5.3.msb.npy']

In [19]:
# delete lnb dir
import shutil
shutil.rmtree('./lnb')